### Create Age column based on DOB column (Age when crime was committed)

In [2]:
import pandas as pd 
import os

In [ ]:
notebook_dir = os.getcwd()
data_dir = os.path.join(notebook_dir, '..', 'data')
checkpoint_dir = os.path.join(data_dir, 'checkpoints')

c:\Users\Indel\Documents\gatewayinitiative-lawrencepd\scripts\..\data\checkpoints


In [8]:
cp11 = pd.read_csv(os.path.join(checkpoint_dir, 'checkpoint11_cleaned_category_copy.csv'))

In [9]:
cp11

,Unnamed: 0,Incident #,Date,Type,Location,Arrested,Location Prefix,DOB,Charges,latitude,longitude,Cleaned Location,person_id,category_archive,Year,crime_severity,category
0,0,18000001.0,2018-01-01 00:01:00,NOISE ORD,3 HARRIMAN ST,Yes,NaN,7/3/1974,A&B ON FAMILY / HOUSEHOLD MEMBER / INTIMATE PA...,42.718522,-71.148148,3 HARRIMAN ST,20f72481f083d4756c89b98fd499514c5953a8a4c10253...,PUBLIC_DISTURBANCES,2018,Non-Serious,Public Disturbances
1,1,18000002.0,2018-01-01 00:08:00,LOUD NOISE,1 HARRIMAN ST FL 2,Yes,NaN,1/21/1979,A&B ON FAMILY / HOUSEHOLD MEMBER / INTIMATE PA...,42.718522,-71.148148,1 HARRIMAN ST,2d5adb553cad8a22fd72d2e390525997c4963bac1b185e...,PUBLIC_DISTURBANCES,2018,Non-Serious,Public Disturbances
2,2,18000003.0,2018-01-01 00:11:00,ALARM/BURG,16 ALLEN ST,No,MATOS,NaN,NaN,42.710782,-71.151911,16 ALLEN ST,NaN,FIRE_AND_ARSON_INCIDENTS,2018,Non-Serious,Fire and Arson Incidents
3,3,18000004.0,2018-01-01 00:14:00,DISORDERLY,11 SUMMER ST,No,NaN,NaN,NaN,42.711117,-71.153015,11 SUMMER ST,NaN,PUBLIC_DISTURBANCES,2018,Non-Serious,Public Disturbances
4,4,18000005.0,2018-01-01 00:27:00,EXTRA SURVEIL,57 SPRINGFIELD ST,Yes,WARD SIX CLUB,6/26/2002,A&B DOMESTIC NO 209A IN EFFECT,42.699339,-71.156938,57 SPRINGFIELD ST,41c43f9f4255dee8e2c910e87f2a983e1b13beecf27eac...,PREVENTIVE_POLICING,2018,Non-Serious,Preventive Policing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424268,424268,24055626.0,2024-12-31 23:11:00,ALARM/BURG,262 HAMPSHIRE ST,No,EL PILON SUPERMARKET,NaN,NaN,42.711212,-71.168891,262 HAMPSHIRE ST,NaN,FIRE_AND_ARSON_INCIDENTS,2024,Non-Serious,Fire and Arson Incidents
424269,424269,24055627.0,2024-12-31 23:19:00,MV/BLOCKING,209 HIGH ST,No,NaN,NaN,NaN,42.713523,-71.149811,209 HIGH ST,NaN,MOTOR_VEHICLE_INCIDENTS,2024,Non-Serious,Motor Vehicle Incidents
424270,424270,24055628.0,2024-12-31 23:23:00,FIRE WORKS,64 BUSWELL ST,No,NaN,NaN,NaN,42.718046,-71.166347,64 BUSWELL ST,NaN,PUBLIC_DISTURBANCES,2024,Non-Serious,Public Disturbances
424271,424271,24055629.0,2024-12-31 23:26:00,FIRE WORKS,ACTON ST & WASHINGTON ST,No,NaN,NaN,NaN,42.710679,-71.174674,ACTON ST & WASHINGTON ST,NaN,PUBLIC_DISTURBANCES,2024,Non-Serious,Public Disturbances


### Drop first column, which is an unnecessary index column

In [14]:
cp11 = cp11.drop(cp11.columns[0], axis=1)
print(cp11.columns.tolist())

['Date', 'Type', 'Location', 'Arrested', 'Location Prefix', 'DOB', 'Charges', 'latitude', 'longitude', 'Cleaned Location', 'person_id', 'category_archive', 'Year', 'crime_severity', 'category']


In [21]:
# work on a copy
df = cp11.copy()

# Optional, tidy column names and Arrested values
df.columns = df.columns.str.strip()
if 'Arrested' in df.columns:
    df['Arrested'] = df['Arrested'].astype(str).str.strip().str.title()

# Keep originals for debugging
df['_DOB_raw'] = df['DOB'].astype(str)
df['_Date_raw'] = df['Date'].astype(str)

# Parse to datetime
df['DOB_dt'] = pd.to_datetime(df['DOB'], errors='coerce', utc=False)
df['Date_dt'] = pd.to_datetime(df['Date'], errors='coerce', utc=False)

# Summary of parsing results
dob_total = df['DOB'].shape[0]
dob_nonnull = df['DOB'].notna().sum()
dob_failed = df['DOB_dt'].isna() & df['DOB'].notna()
date_failed = df['Date_dt'].isna() & df['Date'].notna()

print("DOB non-null count:", dob_nonnull)
print("DOB failed to parse count:", dob_failed.sum())
print("Date failed to parse count:", date_failed.sum())

# Show a few examples of bad DOB and Date entries
if dob_failed.any():
    print("\nSample problematic DOB values:")
    print(df.loc[dob_failed, '_DOB_raw'].drop_duplicates().head(20).to_list())

if date_failed.any():
    print("\nSample problematic Date values:")
    print(df.loc[date_failed, '_Date_raw'].drop_duplicates().head(20).to_list())

# If you want to enforce the standardized columns back into cp11
# only do this once you are happy with the parse results
cp11['DOB'] = df['DOB_dt']
cp11['Date'] = df['Date_dt']

# Final sanity check
print("\nDtypes after standardization:")
print(cp11[['DOB', 'Date']].dtypes)


DOB non-null count: 5007
DOB failed to parse count: 0
Date failed to parse count: 0

Dtypes after standardization:
DOB     datetime64[ns]
Date    datetime64[ns]
dtype: object


In [22]:
import numpy as np

In [23]:
# mask for rows we (those who were charged, assume charged if DOB is present)
mask_charged = cp11['DOB'].notna()

# compute age at offense date with birthday adjustment
before_birthday = (
    (cp11.loc[mask_charged, 'Date'].dt.month < cp11.loc[mask_charged, 'DOB'].dt.month) |
    (
        (cp11.loc[mask_charged, 'Date'].dt.month == cp11.loc[mask_charged, 'DOB'].dt.month) &
        (cp11.loc[mask_charged, 'Date'].dt.day   <  cp11.loc[mask_charged, 'DOB'].dt.day)
    )
)

In [25]:
print(before_birthday)

0          True
1          True
4          True
17         True
20         True
          ...  
424127    False
424181    False
424215    False
424220    False
424221    False
Length: 5007, dtype: bool


In [24]:
age_years = (
    cp11.loc[mask_charged, 'Date'].dt.year - cp11.loc[mask_charged, 'DOB'].dt.year
    - before_birthday.astype(int)
)

In [26]:
print(age_years)

0         43
1         38
4         15
17        30
20        23
          ..
424127    31
424181    34
424215    18
424220    36
424221    39
Length: 5007, dtype: int64


In [27]:
# Write Age, NaN where not charged per your rule
cp11['Age'] = np.nan
cp11.loc[mask_charged, 'Age'] = age_years

In [ ]:
# Quick peek
print(cp11[['DOB','Date','Age']].head(10))
print(cp11['Age'].describe())


         DOB                Date   Age
0 1974-07-03 2018-01-01 00:01:00  43.0
1 1979-01-21 2018-01-01 00:08:00  38.0
2        NaT 2018-01-01 00:11:00   NaN
3        NaT 2018-01-01 00:14:00   NaN
4 2002-06-26 2018-01-01 00:27:00  15.0
5        NaT 2018-01-01 00:28:00   NaN
6        NaT 2018-01-01 00:38:00   NaN
7        NaT 2018-01-01 00:42:00   NaN
8        NaT 2018-01-01 00:44:00   NaN
9        NaT 2018-01-01 01:03:00   NaN
count    5007.000000
mean       33.514280
std        11.151973
min        14.000000
25%        25.000000
50%        32.000000
75%        40.000000
max        84.000000
Name: Age, dtype: float64


In [36]:
valid_age_df = cp11.loc[cp11['Age'].notna(), ['DOB', 'Date', 'Age']]

print(valid_age_df.head(20))   # just a preview

           DOB                Date   Age
0   1974-07-03 2018-01-01 00:01:00  43.0
1   1979-01-21 2018-01-01 00:08:00  38.0
4   2002-06-26 2018-01-01 00:27:00  15.0
17  1987-03-10 2018-01-01 01:41:00  30.0
20  1994-05-27 2018-01-01 02:01:00  23.0
39  1982-06-19 2018-01-01 03:20:00  35.0
40  1991-08-08 2018-01-01 03:20:00  26.0
41  1990-11-18 2018-01-01 03:20:00  27.0
42  1987-05-29 2018-01-01 03:20:00  30.0
57  1978-05-23 2018-01-01 05:05:00  39.0
69  1989-04-26 2018-01-01 06:42:00  28.0
70  1983-12-11 2018-01-01 06:42:00  34.0
93  1994-06-11 2018-01-01 10:29:00  23.0
153 1972-03-01 2018-01-01 19:09:00  45.0
154 1997-05-18 2018-01-01 19:09:00  20.0
172 1968-09-30 2018-01-01 21:24:00  49.0
173 1992-08-13 2018-01-01 21:24:00  25.0
174 1979-01-07 2018-01-01 21:25:00  38.0
175 1987-07-20 2018-01-01 21:25:00  30.0
177 1989-12-01 2018-01-01 21:36:00  28.0


In [37]:
# save path
save_path = os.path.join(checkpoint_dir, 'checkpoint12_age.csv')
cp11.to_csv(save_path, index=False)
print(f"Saved with Age column to: {save_path}")

Saved with Age column to: c:\Users\Indel\Documents\gatewayinitiative-lawrencepd\scripts\..\data\checkpoints\checkpoint12_age.csv
